# Breadth First Search (BFS) 

As the name implies, BFS traverses the given graph in a breadth first manner. Starting at a specified vertex, the algorithms iteratively searches neighboring vertices.  

The algorithm returns, for each vertex:
* the BFS distance (i.e. the number of hops to reach that vertex from starting vertex) 
* the predecessor of the node (i.e. the vertex it was reached from during the search)

@see https://en.wikipedia.org/wiki/Breadth-first_search


Let's get started
___

In [ ]:
# First step is always to import the needed libraries
import cugraph
import cudf
from cudf.dataframe import DataFrame
from collections import OrderedDict

import numpy as np
from scipy.io import mmread

import os.path

# Using read_csv from cuDF

In [ ]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

In [ ]:
# Make sure that the datasets are available
if not os.path.isfile(datafile):
    print("Data File NOT found")

In [ ]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()), )


In [ ]:
# Let's look at the DataFrame. There should be two columns and 154 records
gdf

In [ ]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

In [ ]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

In [ ]:
# How many vertices are in the graph?
G.num_vertices()

In [ ]:
# Call BFS on the graph starting from vertex 0
df = cugraph.bfs(G,0)

In [ ]:
# Use the BFS predecessors and distance to trace the path from vertex
# 22 back to the starting vertex 0:
dist = df['distance'][22]
lastVert = 22
for i in range(dist):
    nextVert = df['predecessor'][lastVert]
    print("Vertex: " + str(lastVert) + " was reached from vertex " + str(nextVert))
    lastVert = nextVert

# As a Matrix file using scipy

In [ ]:
# Test file  - using the clasic Karate club dataset.  
mmFile='../data/networks/karate.mtx'

In [ ]:
# Make sure that the datasets are available
import os.path
import subprocess

if not os.path.isfile(mmFile):
    print("Data File NOT found")

In [ ]:
# Read in the data file into scipy matrix format

M = mmread(mmFile).asfptype().tolil()
M = M.tocsr()

In [ ]:
# Load the structure of the graph into GPU memory and create a CuGraph
# graph object:
row_offsets = cudf.Series(M.indptr)
col_indices = cudf.Series(M.indices)
values = cudf.Series(M.data)

G = cugraph.Graph()
G.add_adj_list(row_offsets, col_indices, values)

In [ ]:
# Call BFS on the graph starting from vertex 0
df = cugraph.bfs(G,0)

In [ ]:
# Use the BFS predecessors and distance to trace the path from vertex
# 22 back to the starting vertex 0:
dist = df['distance'][22]
lastVert = 22
for i in range(dist):
    nextVert = df['predecessor'][lastVert]
    print("Vertex: " + str(lastVert) + " was reached from vertex " + str(nextVert))
    lastVert = nextVert

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___